# Skin Cancer Detection using Machine Learning and Computer Vision

## Project Description

This project aims to develop a robust and accurate skin cancer detection system using machine learning and computer vision techniques. By leveraging a dataset of skin lesion images, the system will be trained to classify images into different categories of skin cancer, aiding in early diagnosis and treatment. The project involves data preprocessing, feature extraction, model training, and evaluation to ensure high performance and reliability.

In [38]:
# Étape 1 : Installer les bibliothèques nécessaires
%pip install tensorflow
%pip install matplotlib
%pip install numpy

import tensorflow as tf
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.applications import EfficientNetB0 # type: ignore #ingore the warning
from tensorflow.keras.layers import Concatenate, Input, Dense, Dropout, Flatten, GlobalAveragePooling2D # type: ignore #ingore the warning
from tensorflow.keras.models import Model, Sequential # type: ignore #ingore the warning
from tensorflow.keras.optimizers import Adam # type: ignore #ingore the warning
from tensorflow.keras.utils import plot_model # type: ignore #ingore the warning
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore #ingore the warning
from tensorflow.keras.applications.efficientnet import preprocess_input # type: ignore #ingore the warning



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [39]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
#If TPU not found try with GPUs
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
else:
  strategy = tf.distribute.get_strategy()
  print('Running on CPU')

# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Running on CPU
Number of accelerators:  1


In [40]:
# TRAINING_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords/train*')
# TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords/test*')
BATCH_SIZE = 10 * strategy.num_replicas_in_sync
IMAGE_SIZE = [100, 100]
AUTO = tf.data.experimental.AUTOTUNE
imSize = 100
EPOCHS = 10

# Définir l'entrée du modèle
input_layer = Input(shape=(imSize, imSize, 3))

VALIDATION_SPLIT = 0.18
# split = int(len(TRAINING_FILENAMES) * VALIDATION_SPLIT)
# training_filenames = TRAINING_FILENAMES[split:]
# validation_filenames = TRAINING_FILENAMES[:split]
# print("Pattern matches {} data files. Splitting dataset into {} training files and {} validation files"
#       .format(len(TRAINING_FILENAMES), len(training_filenames), len(validation_filenames)))
# TRAINING_FILENAMES = training_filenames

In [41]:
# Charger le modèle EfficientNetB0 préentraîné sans la couche de classification finale
with strategy.scope():
    base_model = tf.keras.Sequential([
        EfficientNetB0(
            input_shape=(imSize, imSize, 3),
            weights='imagenet',
            include_top=False
        )
    ])


# Geler les couches du modèle de base pour conserver les poids préentraînés
base_model.trainable = False




In [ ]:
# Passer l'entrée à travers le modèle de base
x = base_model(input_layer, training=False)  # `input_layer` 
x = Flatten()(x)  # Aplatir les caractéristiques extraites


# Ajout de couches fully connected pour la classification
x = Dense(128, activation="relu")(x)  # Couche dense
x = Dropout(0.5)(x)  # Dropout pour régularisation
output = Dense(1, activation="sigmoid")(x)  # Sigmoid pour une classification binaire

# Définir le modèle final avec une seule entrée (image)
model = Model(inputs=input_layer, outputs=output)


NameError: name 'layers' is not defined

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimiseur Adam
    loss="binary_crossentropy",          # Fonction de perte pour la classification binaire
    metrics=["accuracy"]                 # Suivi de la précision
)

# Résumé du modèle
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_7 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
# Visualisation de l'architecture du modèle
plot_model(model, show_shapes=True, to_file="efficientnetb0_model.png")

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
# Fonction pour charger les matrices Y, Cb, Cr à partir du fichier
def load_ycbcr_from_file(file_path):
    """
    Charger les matrices Y, Cb, Cr à partir d'un fichier texte.
    Les fichiers doivent contenir trois sections avec des matrices pour chaque composante.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Identifier les sections et extraire les matrices
    y_start = lines.index("Matrice de luminance (Y):\n") + 1
    cb_start = lines.index("Matrice de chrominance (Cb):\n") + 1
    cr_start = lines.index("Matrice de chrominance (Cr):\n") + 1

    y = np.loadtxt(lines[y_start:cb_start - 1], dtype=np.float32)
    cb = np.loadtxt(lines[cb_start:cr_start - 1], dtype=np.float32)
    cr = np.loadtxt(lines[cr_start:], dtype=np.float32)

    return y, cb, cr

In [ ]:
# Fonction pour préparer l'entrée pour le modèle
def prepare_ycbcr_input(y, cb, cr):
    """
    Prépare l'entrée pour le modèle à partir des matrices Y, Cb, Cr.
    """
    # Empiler les canaux Y, Cb, Cr pour former une image (100, 100, 3)
    image = np.stack([y, cb, cr], axis=-1)  # Shape: (100, 100, 3)

    # Normaliser l'image (en utilisant une normalisation simple, pas celle d'ImageNet)
    image = image / 255.0  # Normalisation simple entre 0 et 1

    # Ajouter la dimension du batch (1, 100, 100, 3)
    image = np.expand_dims(image, axis=0)

    return image

In [ ]:
# Fonction de prédiction
def predict_skin_cancer(file_path):
    """
    Prédire à partir des matrices Y, Cb, Cr chargées depuis un fichier.
    """
    # Charger les matrices Y, Cb, Cr à partir du fichier
    y, cb, cr = load_ycbcr_from_file(file_path)

    # Préparer l'entrée pour le modèle
    ycbcr_input = prepare_ycbcr_input(y, cb, cr)


    # Prédiction
    prediction = model.predict(ycbcr_input)

    # Interprétation
    if prediction[0] > 0.5:
        return f"Maligne (probabilité {prediction[0][0]:.2f})"
    else:
        return f"Bénigne (probabilité {1 - prediction[0][0]:.2f})"

# Exemple d'utilisation
file_path = "./DataBase/Matrices/ISIC_0001120_matrices.txt"
result = predict_skin_cancer(file_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Bénigne (probabilité 0.52)
